In [1]:
pip install realtabformer

  Using cached realtabformer-0.2.2-py3-none-any.whl.metadata (11 kB)
  Using cached datasets-3.3.2-py3-none-any.whl.metadata (19 kB)
  Using cached shapely-2.0.7-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiohttp-3.11.13-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.7 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2024.11.6-cp312-cp312-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.7 kB

In [2]:
import pandas as pd
ads_df_minimal = pd.read_csv("train_ads_minimal_user.csv")
feeds_df_minimal = pd.read_csv("train_feeds_minimal_user.csv")

In [3]:
ads_df_minimal.shape, feeds_df_minimal.shape

((7675517, 18), (3227732, 16))

In [4]:
feeds_df_minimal.rename(columns={'u_userId': 'user_id'}, inplace=True)

In [5]:
ads_df_minimal.columns, feeds_df_minimal.columns

(Index(['label', 'user_id', 'age', 'gender', 'residence', 'city_rank',
        'series_dev', 'series_group', 'emui_dev', 'net_type', 'creat_type_cd',
        'inter_type_cd', 'slot_id', 'hispace_app_tags', 'app_second_class',
        'app_score', 'u_refreshTimes', 'u_feedLifeCycle'],
       dtype='object'),
 Index(['user_id', 'u_phonePrice', 'u_browserLifeCycle', 'u_browserMode',
        'u_feedLifeCycle', 'u_refreshTimes', 'i_dislikeTimes', 'i_upTimes',
        'i_dtype', 'e_ch', 'e_po', 'e_rn', 'e_section', 'label', 'cillabel',
        'pro'],
       dtype='object'))

In [6]:
ads_df_minimal['label'].value_counts(), feeds_df_minimal['label'].value_counts()

(label
 0    7556381
 1     119136
 Name: count, dtype: int64,
 label
 -1    2911971
  1     315761
 Name: count, dtype: int64)

In [7]:
ads_df_minimal_label1 = ads_df_minimal[ads_df_minimal['label']==1]
feeds_df_minimal_label1 = feeds_df_minimal[feeds_df_minimal['label']==1]

In [8]:
ads_df_minimal_label1['label'].value_counts(), feeds_df_minimal_label1['label'].value_counts()

(label
 1    119136
 Name: count, dtype: int64,
 label
 1    315761
 Name: count, dtype: int64)

In [9]:
ads_df_minimal_label1.to_csv('ads_df_minimal_label1.csv', index=False)
feeds_df_minimal_label1.to_csv('feeds_df_minimal_label1.csv', index=False)

In [12]:
ads_df_minimal_label1.shape, feeds_df_minimal_label1.shape

((119136, 18), (315761, 16))

In [ ]:
import pandas as pd
import os
from pathlib import Path
from realtabformer import REaLTabFormer


join_on = "user_id"  # This is the common key in both datasets

# ----- Parent Modeling (Ads) -----
# Remove the unique identifier from features to be modeled
ads_model = REaLTabFormer(model_type="tabular")
ads_model.fit(ads_df_minimal.drop(join_on, axis=1))
ads_model.save("rtf_ads/")

# Retrieve the most recent saved model directory for the parent
ads_model_path = sorted([
    p for p in Path("rtf_ads/").glob("id*") if p.is_dir()
], key=os.path.getmtime)[-1]

# ----- Child Modeling (Feeds) -----
feeds_model = REaLTabFormer(
    model_type="relational",
    parent_realtabformer_path=ads_model_path,
    train_size=0.8
)

# Fit the relational model using both the child and parent data
feeds_model.fit(
    df=feeds_df_minimal,
    in_df=ads_df_minimal,
    join_on=join_on
)

# ----- Generating Synthetic Data -----
# Generate synthetic ads data from the parent model
synthetic_ads = ads_model.sample(n_samples=len(ads_df_minimal))
# Ensure the synthetic ads data contains the join key (or recreate it as needed)
synthetic_ads.index.name = join_on
synthetic_ads = synthetic_ads.reset_index()

# Generate synthetic feeds data using the relational model
synthetic_feeds = feeds_model.sample(
    input_unique_ids=synthetic_ads[join_on],
    input_df=synthetic_ads.drop(join_on, axis=1),
    gen_batch=64
)

/usr/local/lib/python3.11/dist-packages/realtabformer/realtabformer.py:77: UserWarning: The device=cuda is not available, using device=cpu instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/realtabformer/realtabformer.py:536: UserWarning: The frac (0.165) set results to a sample larger than                     frac_max_data=10000. Setting frac to 0.001302843834493494.
  warnings.warn(


Computing the sensitivity threshold...
Using parallel computation!!!


ate (0.7664892149935959) in                     the data. This will not give a reliable early stopping condition. Consider                         using qt_max="compute" argument.
  warnings.warn(


Bootstrap round:   0%|          | 0/500 [00:00<?, ?it/s]

Sensitivity threshold summary:
count    500.000000
mean       0.037246
std        0.001661
min        0.033011
25%        0.036250
50%        0.037246
75%        0.038325
max        0.043021
dtype: float64
Sensitivity threshold: 0.039935252259644886 qt_max: 0.05


In [10]:
synthetic_ads

NameError: name 'synthetic_ads' is not defined

In [ ]:
import pandas as pd
import os
from pathlib import Path
from realtabformer import REaLTabFormer

# ---------------------------
# 1. Load Your Datasets
# ---------------------------
# Replace these file paths with your actual CSV files.
ads_df = pd.read_csv("ads_df_minimal_label1.csv")    # Contains only label 1 rows for ads
feeds_df = pd.read_csv("feeds_df_minimal_label1.csv")  # Contains only label 1 rows for feeds

# ---------------------------
# 2. Train the Parent Model (Ads Data)
# ---------------------------
# Since "user_id" is used for joining, remove it from the features to be modeled.
parent_features = ads_df.drop(columns=["user_id"])

# Initialize and fit the parent model in tabular mode.
parent_model = REaLTabFormer(model_type="tabular",
                             gradient_accumulation_steps=4,
                             logging_steps=100)
parent_model.fit(parent_features)

# Save the parent model (this creates a folder with a unique experiment id)
parent_model.save("rtf_ads/")

# Retrieve the path of the saved parent model (assumes the most recent is the one we need)
parent_model_path = sorted([p for p in Path("rtf_ads/").glob("id*") if p.is_dir()],
                           key=os.path.getmtime)[-1]

# ---------------------------
# 3. Train the Child Model (Feeds Data)
# ---------------------------
# The child (relational) model uses the feeds data.
# It also takes the original ads data for relational joining via "user_id."
child_model = REaLTabFormer(model_type="relational",
                            parent_realtabformer_path=parent_model_path,
                            train_size=0.8)  # You can adjust train_size as needed

# Fit the child model.
child_model.fit(
    df=feeds_df,
    in_df=ads_df,    # The parent table used for the join
    join_on="user_id"
)

# ---------------------------
# 4. Generate Synthetic Samples
# ---------------------------
# Generate synthetic parent samples (for ads data)
synthetic_ads = parent_model.sample(n_samples=len(ads_df))

# Since we dropped "user_id" before training, you might need to reassign or generate new ids.
# Here, we simply reset the index to create a new "user_id" column.
synthetic_ads.index.name = "user_id"
synthetic_ads = synthetic_ads.reset_index()

# Generate synthetic child samples (for feeds data) using the relational model.
synthetic_feeds = child_model.sample(
    input_unique_ids=synthetic_ads["user_id"],
    input_df=synthetic_ads.drop(columns=["user_id"]),
    gen_batch=64  # Adjust batch size if needed
)

# ---------------------------
# 5. Display Sample Outputs
# ---------------------------
print("Synthetic Ads Samples:")
print(synthetic_ads.head())

print("\nSynthetic Feeds Samples:")
print(synthetic_feeds.head())


/Users/davidliu/Desktop/realtabformer Project/newprojectenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/davidliu/Desktop/realtabformer Project/newprojectenv/lib/python3.12/site-packages/realtabformer/realtabformer.py:77: UserWarning: The device=cuda is not available, using device=mps instead.
  warnings.warn(
/Users/davidliu/Desktop/realtabformer Project/newprojectenv/lib/python3.12/site-packages/realtabformer/realtabformer.py:536: UserWarning: The frac (0.165) set results to a sample larger than                     frac_max_data=10000. Setting frac to 0.08393768466290626.
  warnings.warn(
ate (0.2197236771420897) in                     the data. This will not give a reliable early stopping condition. Consider                         using qt_max="compute" argument.uplicate 
  warnings.warn(


Computing the sensitivity threshold...
Using parallel computation!!!


Bootstrap round: 100%|██████████| 500/500 [28:22<00:00,  3.41s/it]


Sensitivity threshold summary:
count    500.000000
mean       0.049615
std        0.001666
min        0.042790
25%        0.048437
50%        0.049663
75%        0.050703
max        0.054119
dtype: float64
Sensitivity threshold: 0.05225801965582736 qt_max: 0.05


Map: 100%|██████████| 119136/119136 [00:12<00:00, 9885.74 examples/s] 
/Users/davidliu/Desktop/realtabformer Project/newprojectenv/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


...

Generated 0 invalid samples out of total 30080 samples generated. Sampling efficiency is: 100.0000%


/Users/davidliu/Desktop/realtabformer Project/newprojectenv/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Critic round: 10,                     sensitivity_threshold: 0.05225801965582736,                         val_sensitivity: 0.0033107662157731014,                             val_sensitivities: [np.float64(0.0019003712494674698), np.float64(0.0035986460740200254), np.float64(0.002618931743521109), np.float64(0.0022241127856101133), np.float64(0.002574461182863286), np.float64(0.004836225068915194), np.float64(0.004039327894800925), np.float64(0.0035963630459490176), np.float64(0.004610073111291633), np.float64(0.0031387039142439493), np.float64(0.0030192062237889743), np.float64(0.0031569965870307153), np.float64(0.00352621327706541), np.float64(0.00236474836196929), np.float64(0.0044571127160594)]


Step,Training Loss
37300,0.595700
37400,0.593100
37500,0.591800
37600,0.595700
37700,0.596000
37800,0.596100
37900,0.594300
38000,0.593200
38100,0.595600
38200,0.597200


KeyboardInterrupt: 

In [13]:
len(ads_df)

119136

In [16]:
import pandas as pd
import os
from pathlib import Path
from realtabformer import REaLTabFormer

# ---------------------------
# 1. Load Your Datasets
# ---------------------------
# Replace these file paths with your actual CSV files.
ads_df = pd.read_csv("ads_df_minimal_label1.csv")    # Contains only label 1 rows for ads
feeds_df = pd.read_csv("feeds_df_minimal_label1.csv")  # Contains only label 1 rows for feeds

# ---------------------------
# 2. Train the Parent Model (Ads Data)
# ---------------------------
# Since "user_id" is used for joining, remove it from the features to be modeled.
parent_features = ads_df.drop(columns=["user_id"])

# Initialize and fit the parent model in tabular mode.
parent_model = REaLTabFormer(model_type="tabular",
                             gradient_accumulation_steps=8,
                             epochs=50, 
                             batch_size = 128,
                             logging_steps=500,
                             early_stopping_threshold=0.01,
                             early_stopping_patience=3
                             )

parent_model.fit(parent_features)

# Save the parent model (this creates a folder with a unique experiment id)
parent_model.save("rtf_ads/")

# Retrieve the path of the saved parent model (assumes the most recent is the one we need)
parent_model_path = sorted([p for p in Path("rtf_ads/").glob("id*") if p.is_dir()],
                           key=os.path.getmtime)[-1]

# ---------------------------
# 3. Train the Child Model (Feeds Data)
# ---------------------------
# The child (relational) model uses the feeds data.
# It also takes the original ads data for relational joining via "user_id."
child_model = REaLTabFormer(model_type="relational",
                            parent_realtabformer_path=parent_model_path,
                            train_size=0.8)  # You can adjust train_size as needed

# Fit the child model.
child_model.fit(
    df=feeds_df,
    in_df=ads_df,    # The parent table used for the join
    join_on="user_id"
)

# ---------------------------
# 4. Generate Synthetic Samples
# ---------------------------
# Generate synthetic parent samples (for ads data)
synthetic_ads = parent_model.sample(n_samples=len(ads_df))

# Since we dropped "user_id" before training, you might need to reassign or generate new ids.
# Here, we simply reset the index to create a new "user_id" column.
synthetic_ads.index.name = "user_id"
synthetic_ads = synthetic_ads.reset_index()

# Generate synthetic child samples (for feeds data) using the relational model.
synthetic_feeds = child_model.sample(
    input_unique_ids=synthetic_ads["user_id"],
    input_df=synthetic_ads.drop(columns=["user_id"]),
    gen_batch=64  # Adjust batch size if needed
)

# ---------------------------
# 5. Display Sample Outputs
# ---------------------------
print("Synthetic Ads Samples:")
print(synthetic_ads.head())

print("\nSynthetic Feeds Samples:")
print(synthetic_feeds.head())


/Users/davidliu/Desktop/realtabformer Project/newprojectenv/lib/python3.12/site-packages/realtabformer/realtabformer.py:77: UserWarning: The device=cuda is not available, using device=mps instead.
  warnings.warn(
/Users/davidliu/Desktop/realtabformer Project/newprojectenv/lib/python3.12/site-packages/realtabformer/realtabformer.py:536: UserWarning: The frac (0.165) set results to a sample larger than                     frac_max_data=10000. Setting frac to 0.08393768466290626.
  warnings.warn(
ate (0.2197236771420897) in                     the data. This will not give a reliable early stopping condition. Consider                         using qt_max="compute" argument.uplicate 
  warnings.warn(


Computing the sensitivity threshold...
Using parallel computation!!!








































































































































































Bootstrap round: 100%|██████████| 500/500 [28:02<00:00,  3.37s/it]


Sensitivity threshold summary:
count    500.000000
mean       0.049724
std        0.001777
min        0.044984
25%        0.048514
50%        0.049688
75%        0.050957
max        0.053720
dtype: float64
Sensitivity threshold: 0.05274019980209938 qt_max: 0.05









































































































Map: 100%|██████████| 119136/119136 [00:13<00:00, 9159.29 examples/s]
/Users/davidliu/Desktop/realtabformer Project/newprojectenv/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Generated 0 invalid samples out of total 30080 samples generated. Sampling efficiency is: 100.0000%


/Users/davidliu/Desktop/realtabformer Project/newprojectenv/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Critic round: 25,                     sensitivity_threshold: 0.05274019980209938,                         val_sensitivity: 0.00407718005231483,                             val_sensitivities: [np.float64(0.004411764705882354), np.float64(0.005282132933087236), np.float64(0.0030817884627099014), np.float64(0.00391356323702233), np.float64(0.004601995093367937), np.float64(0.005073968993819029), np.float64(0.005075035489758671), np.float64(0.0039019687436574), np.float64(0.004532353657546481), np.float64(0.0052181633688483005), np.float64(0.0033906554185593753), np.float64(0.0030028429282160625), np.float64(0.002226767912614458), np.float64(0.0041516208869233585), np.float64(0.00329307895270956)]


Step,Training Loss
3000,0.597900


KeyboardInterrupt: 

In [ ]:
synthetic_ads.to_csv('synthetic_ads.csv', index=False)
synthetic_feeds.to_csv('synthetic_feeds.csv', index=False)

NameError: name 'synthetic_ads' is not defined